## CRF used for Categorizing Sentences Based on Methods Used to commit Suicide

### Objective : Applying CRF to Extract Patterns/Statistics in Sentences with Methods and Predicting Labels

#### Importing Required Packages

In [36]:
import pandas as pd
import numpy as np
import csv
import io
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import scipy.stats

from collections import Counter
import math, re
from itertools import zip_longest
from datetime import datetime
from functools import reduce

import sklearn
print(sklearn.__version__)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics
from sklearn import model_selection
from sklearn.externals import joblib
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

import re

import sklearn_crfsuite

import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import sent_tokenize, word_tokenize, pos_tag

import pprint



0.18.1


#### Getting all the Sentences in a Dataframe

In [37]:
fileallpostsannotated = 'C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Manual_Annotations_Sent.csv'
allannotations =  pd.read_csv(fileallpostsannotated,index_col=None, sep="|", header = None)
allannotations.columns = ['sentence','annotation']

In [38]:
allannotations.head()

,sentence,annotation
0,Well I had attempted to hang myself when I was...,hang
1,So I got myself all strung up by a belt stuck ...,hang
2,"I hanged for about fifteen seconds, just long ...",hang
3,And then the cheap plastic belt I was using br...,hang
4,"The belt broke and I was still here, and 2.",hang


#### We defined a few dictionaries from common occurrences of words... So, let's get those dictionaries

In [39]:
ability_verbs = list()
FPRP = list()
WPRP = list()
SPRP = list()
swear = list()
POSEmo = list()
intensifier = list()
negative = list()
positive = list()
negations = list()
methods = list()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Final_Methods.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    methods.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/abilityverbs.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    ability_verbs.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/FPRP.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    FPRP.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/WPRP.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    WPRP.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/SPRP.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    SPRP.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/All_Negative_Words.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    negative.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/Negations.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    negations.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/swear.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    swear.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/POSEmo.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    POSEmo.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/extreme_intensifiers_level0.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    intensifier.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/extreme_intensifiers_level1.txt', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    intensifier.append(row[0])
csvfile.close()

csvfile = open('C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionaries/All_Postive_Words.csv', 'r', encoding='utf8')
reader = csv.reader(csvfile, delimiter='|')
for row in reader:
    positive.append(row[0])
csvfile.close()

methods = [w.replace(w, w.lower()) for w in methods]
ability_verbs = [w.replace(w, w.lower()) for w in ability_verbs]
FPRP = [w.replace(w, w.lower()) for w in FPRP]
WPRP = [w.replace(w, w.lower()) for w in WPRP]
SPRP = [w.replace(w, w.lower()) for w in SPRP]
swear = [w.replace(w, w.lower()) for w in swear]
POSEmo = [w.replace(w, w.lower()) for w in POSEmo]
intensifier = [w.replace(w, w.lower()) for w in intensifier]
negative = [w.replace(w, w.lower()) for w in negative]
positive = [w.replace(w, w.lower()) for w in positive]
negations = [w.replace(w, w.lower()) for w in negations]

In [40]:
textposts = allannotations.sentence 
textposts_postagged= list()
textposts_postagged = [nltk.word_tokenize(sent) for sent in textposts]
textposts_postagged = [nltk.pos_tag(sent) for sent in textposts_postagged]

In [41]:
posts = list()
for sent in textposts:
    posts.append(sent)

#### Using Vader Lexicon and getting it's sentiment scores

In [42]:
#positive sentiment: compound score >= 0.5
#neutral sentiment: (compound score > -0.5) and (compound score < 0.5)
#negative sentiment: compound score <= -0.5
Vader_emotions = list()
senti_Vader = list()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
for sentence in posts:
    ss = sid.polarity_scores(sentence)
    senti_Vader.append(ss['compound'])

In [43]:
for i, value in enumerate(senti_Vader):
    if value<0.5 and value> -0.5:
        senti_Vader[i] = 0
    elif value>= 0.5 and value<0.6:
        senti_Vader[i] = 1
    elif value>= 0.6 and value<0.7:
        senti_Vader[i] = 2
    elif value>= 0.7 and value<0.8:
        senti_Vader[i] = 3
    elif value>= 0.8 and value<0.9:
        senti_Vader[i] = 4
    elif value >= 0.9 and value < 1:
        senti_Vader[i] = 5
    elif value <= -0.5 and value > -0.6:
        senti_Vader[i] = -1
    elif value <= -0.6 and value> -0.7:
        senti_Vader[i] = -2
    elif value <= -0.7 and value> -0.8:
        senti_Vader[i] = -3
    elif value <= -0.8 and value> -0.9:
        senti_Vader[i] = -4
    elif value <= -0.9 and value> -1:
        senti_Vader[i] = -5
    

#### Using Afinn Lexicon

In [44]:
from afinn import Afinn
afinn = Afinn(emoticons=True)
senti_Afinn = list()
for sentence in posts:
    senti_Afinn.append(afinn.score(sentence))

#### Getting Simple Freq features from Sentences using the dictionaries we have created

In [45]:
scored_sentences = list()
for i, sent in enumerate(textposts_postagged):
    cfprp = cwprp = csprp = cintfr = cpositive = cnegative = cablt = cswr = cneg = cnegintfr = flag= cmethod = 0
    for j, word in enumerate(sent):
        if word[0].lower() in FPRP:
            cfprp += 1
        if word[0].lower() in WPRP:
            cwprp += 1
        elif word[0].lower() in SPRP:
            csprp += 1
        elif word[0].lower() in negations:
            cneg += 1
        elif word[0].lower() in ability_verbs:
            cablt += 1
        elif word[0].lower() in swear:
            cswr += 1
        if word[0].lower() in intensifier and word[0].lower() in negative:
            cnegintfr += 1
        elif word[0].lower() in intensifier:
            cintfr += 1
        elif word[0].lower() in negative:
            cnegative += 1
        elif word[0].lower() in positive:
            cpositive += 1
        if word[0].lower() in methods:
            cmethod += 1
    for expr in POSEmo:
        matchObj = re.compile(r"%s" % expr)
        match = matchObj.findall(posts[i])
        if (len(match))>0:
            flag = 1
            break
    scored_sentences.append([posts[i], cfprp, csprp, cneg, cablt, cswr, cintfr, cnegative, cnegintfr, flag, cpositive, cwprp, senti_Vader[i], int(senti_Afinn[i]), cmethod])

#### Function to return wordnet compatible POS Tags

In [46]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

#### Reading the Dictionary Map which is a Map from Words to the Annotations

In [47]:
synonym_dict = dict()
with open("C:/Users/AshwinAmbal/Desktop/Intern_Final/Resources/Dictionary_Map.csv", "r",encoding = 'utf8') as f:
    reader = csv.reader(f, delimiter='|')
    for key in reader:
        synonym_dict[key[0]] = key[1]

In [93]:
for key in synonym_dict:
        if synonym_dict[key] == 'suffocate':
            print(key)

suffocation


#### Function to Get our type of tags from the Sentences

In [48]:
def get_the_tag(word):
        tupled = tuple()
        if word[0].lower() in negations:
            tupled = (word[0], 'NEG')
            get_the_tag.flag = 1
        elif word[0].lower() in SPRP:
            tupled = (word[0], 'SPRP')
        elif word[0].lower() in FPRP:
            tupled = (word[0], 'FPRP')
            get_the_tag.count_PRP += 1
        elif word[0].lower() in WPRP:
            tupled = (word[0], 'WPRP')
        elif word[0].lower() in ability_verbs:
            tupled = (word[0], 'ABLT')
        elif word[0].lower() in positive:
            tupled = (word[0], 'POSITIVE')
        if word[0].lower() in intensifier and word[0].lower() in negative:
            tupled = (word[0], 'NEGINTSFR')
            get_the_tag.count_neg += 1
        elif word[0].lower() in intensifier:
            tupled = (word[0], 'INTSFR')
        elif word[0].lower() in swear:
            tupled = (word[0], 'SWR')
        elif word[0].lower() in negative:
            tupled = (word[0], 'NEGATIVE')
            get_the_tag.count_neg += 1
        temp_word = wordnet_lemmatizer.lemmatize(word[0].lower(), get_wordnet_pos(nltk.pos_tag([word[0].lower()])[0][1]))
        if temp_word in methods:
            tupled = (word[0], synonym_dict[temp_word])
            """ The following 2 lines of code are used to add words to synonym_dict and map words to annotations 
            when we have manually annotated sentences as we will absolutely be sure that the word will 
            be related to that particular annotation"""
            #synonym_dict[temp_word].append(allannotations['annotation'][get_the_tag.sent_no])
            #synonym_dict[temp_word] = allannotations['annotation'][get_the_tag.sent_no]
        for expr in POSEmo:
            matchObj = re.compile(r"%s" % expr)
            match = matchObj.findall(word[0].lower())
            if (len(match))>0:
                tupled = (word[0], 'POSEMO')
                break
        if len(tupled) == 0:
            tupled = (word[0], word[1])
        return tupled

#### Getting our POS Tagges Sentences by querying the function

In [49]:
sentences_ourtag = list()
get_the_tag.sent_no = 0
wordnet_lemmatizer = WordNetLemmatizer()
for i, sent in enumerate(textposts_postagged):
    
    tagger = list()
    get_the_tag.flag = 0
    get_the_tag.count_PRP = 0
    get_the_tag.count_neg = 0
    
    for j, word in enumerate(sent):
        tupled = tuple()
        tupled = get_the_tag(word)
        # For appending the NEG to signify words coming after a Negation
        #if get_the_tag.flag == 1:
        #    get_the_tag.flag += 1
        #elif get_the_tag.flag == 2:
        #    tupled = (tupled[0], tupled[1] + "_NEG")
        
        tagger.append(tupled)
    sentences_ourtag.append(tagger)
    get_the_tag.sent_no += 1

In [59]:
sentences_ourtag

[[('Well', 'POSITIVE'),
  ('I', 'FPRP'),
  ('had', 'VBD'),
  ('attempted', 'VBN'),
  ('to', 'TO'),
  ('hang', 'hang'),
  ('myself', 'FPRP'),
  ('when', 'WRB'),
  ('I', 'FPRP'),
  ('was', 'VBD'),
  ('15', 'CD'),
  ('.', '.')],
 [('So', 'INTSFR'),
  ('I', 'FPRP'),
  ('got', 'VBD'),
  ('myself', 'FPRP'),
  ('all', 'DT'),
  ('strung', 'VBP'),
  ('up', 'RP'),
  ('by', 'IN'),
  ('a', 'DT'),
  ('belt', 'hang'),
  ('stuck', 'NEGATIVE'),
  ('in', 'IN'),
  ('the', 'DT'),
  ('door', 'NN'),
  ('jam', 'NEGATIVE'),
  ('of', 'IN'),
  ('my', 'FPRP'),
  ('closet', 'NN'),
  (',', ','),
  ('stood', 'VBD'),
  ('up', 'RP'),
  ('on', 'IN'),
  ('a', 'DT'),
  ('stool', 'NEGATIVE'),
  (',', ','),
  ('and', 'CC'),
  ('tried', 'VBD'),
  ('to', 'TO'),
  ('get', 'VB'),
  ('the', 'DT'),
  ('courage', 'POSITIVE'),
  ('to', 'TO'),
  ('knock', 'NEGATIVE'),
  ('the', 'DT'),
  ('stool', 'NEGATIVE'),
  ('over', 'IN'),
  ('.', '.')],
 [('I', 'FPRP'),
  ('hanged', 'hang'),
  ('for', 'IN'),
  ('about', 'IN'),
  ('fifteen', 

#### Rules to Categorize Sentences into Methods

##### Note: Put the rules into a CSV File if neatness preferred and read from them

In [60]:
JumpSeqTags =   [
                    ['FPRP', 'jump', 'NEGATIVE'],
                    ['FPRP', 'VBD', 'TO', 'jump'],
                    ['FPRP', 'TO', 'jump'],
                    ['FPRP', 'VBD', 'VBG', 'TO', 'jump'],
                    ['FPRP', 'MD', 'jump'],
                    ['FPRP', 'MD', 'RB', 'jump'],
                    ['jump', 'IN', 'NNS'],
                    ['IN', 'jump', 'RP'],
                    ['jump', 'NEGATIVE', 'DT']
                ]

VehSeqTags = [
                ['FPRP', 'VBD', 'TO', 'vehicle', 'IN', 'NN', 'IN', 'DT', 'NN', 'vehicle'],
                ['FPRP', 'VBD', 'DT', 'NN', 'CC', 'NEGATIVE', 'DT', 'vehicle'],    
                ['FPRP','VBP','IN','DT','NN','NN','VBG','IN','DT','NN','vehicle'],
                ['FPRP', 'VBD', 'vehicle'],
                ['FPRP','VBP','VBG','TO','NEGATIVE','DT','NN','CC','VB','NEGATIVE','RB','IN','DT','vehicle'],
                ['FPRP', 'MD', 'RB', 'vehicle']
              ]


GunSeqTags =   [
                    ['FPRP', 'VBP', 'DT', 'gun'],
                    ['FPRP', 'VBD', 'DT', 'gun'],
                    ['FPRP', 'VBD', 'DT', 'JJ', 'gun'],
                    ['FPRP','VBP','DT','RB','NEGATIVE','NNS','INTSFR','DT','NEGATIVE','IN','DT','gun'],
                    ['FPRP', 'VBD', 'DT', 'NN', 'CC', 'DT', 'gun'],
                    ['FPRP', 'VBD', 'SWR', 'NEGATIVE', 'CC', 'VBD', 'DT', 'gun'],
                    ['FPRP', 'VBD', 'FPRP', 'gun'],
                    ['DT', 'gun', 'VBD', 'NEG', 'NEGATIVE'],
                    ['DT', 'NN', 'IN', 'DT', 'gun'],
                    ['DT', 'NN', 'IN', 'DT', 'gun'],
                    ['DT', 'gun', 'VBD'],
                    ['VB', 'DT', 'gun'],
                    ['VBG', 'DT', 'gun'],
                    ['DT', 'gun', 'TO', 'FPRP', 'NN'],
                    ['gun', 'VBD', 'IN', 'FPRP', 'NN'],
                    ['VBG', 'IN', 'DT', 'gun'],
                    ['FPRP', 'VBD', 'FPRP', 'NNS', 'gun'],
                    ['FPRP', 'VBD', 'INTSFR', 'VBN', 'DT', 'gun'],
                    ['FPRP', 'NN', "''", 'NN', 'gun'],
                    ['TO', 'VB', 'CD', 'gun'],
                    ['FPRP', 'INTSFR', 'VBD', 'DT', 'gun'],
                    ['VBD', 'IN', 'DT', 'gun']
                ]

CutSeqTags =   [
                    ['FPRP', 'VBD', 'TO', 'cut'],
                    ['FPRP', 'VBD', 'IN', 'DT', 'NNP', 'CC', 'VB', 'CD', 'NN', 'IN', 'cut'],
                    ['FPRP', 'VBD', 'TO', 'DT', 'NN', 'CC', 'cut'],
                    ['FPRP', 'MD', 'cut'],
                    ['FPRP', 'VBP', 'NEG', 'INTSFR', 'cut'],
                    ['FPRP', 'VBD', 'DT', 'NN', 'WP', 'cut'],
                    ['FPRP', 'MD', 'VBN', 'cut'],
                    ['DT', 'JJ', 'cut'],
                    ['FPRP', 'VBD', 'cut', 'FPRP'],
                    ['WRB', 'FPRP', 'cut', 'FPRP'],
                    ['FPRP', 'cut', 'FPRP'],
                    ['SPRP', 'ABLT', 'INTSFR', 'cut', 'SPRP'],
                    ['FPRP', 'cut', 'FPRP', 'NNS'],
                    ['DT', 'cut', 'IN', 'FPRP', 'NNS'],
                    ['FPRP', 'VBD', 'cut', 'FPRP', 'NNS'],
                    ['SPRP', 'VBD', 'IN', 'FPRP', 'cut'],
                    ['FPRP', 'VBD', 'SPRP', 'IN', 'cut'],
                    ['cut', 'FPRP', 'NNS'],
                    ['VBN', 'cut', 'NNS'],
                    ['cut', 'FPRP', 'NN'],
                ]

AlcSeqTags =   [
                    ['FPRP', 'VBP', 'alcohol', 'NN'],
                    ['DT', 'NN', 'IN', 'alcohol'],
                    ['FPRP', 'alcohol', 'DT', 'POSITIVE', 'NN', 'IN', 'NN'],
                    ['FPRP', 'VBP', 'VBN', 'alcohol'],
                    ['FPRP', 'VBP', 'VBG', 'NN', 'CC', 'alcohol'],
                    ['FPRP', 'VBP', 'INTSFR', 'NEGATIVE', 'IN', 'DT', 'alcohol'],
                    ['FPRP', 'MD', 'VB', 'alcohol'],
                    ['FPRP', 'VBP', 'CD', 'NEGATIVE', 'NEGATIVE', 'NNS', 'RB', 'IN', 'alcohol'],
                    ['FPRP', 'alcohol', 'SPRP', 'NEGATIVE', 'INTSFR'],
                    ['FPRP', 'VBD', 'alcohol'],
                    ['IN', 'VBG', 'PDT', 'DT', 'alcohol'],
                    ['DT', 'NEGATIVE', 'NNS', 'IN', 'alcohol'],
                    ['alcohol', 'DT', 'IN', 'DT', 'NEGATIVE', 'NN'],
                    [ 'FPRP', 'VBD', 'alcohol', 'PDT', 'DT', 'NN', 'IN', 'NN'],
                    ['FPRP', 'VBD', 'JJ', 'TO', 'alcohol'],
                    ['FPRP', 'VBD', 'VBN', 'alcohol'],
                    ['VBD', 'SPRP', 'IN', 'alcohol'],
                    ['FPRP', 'POSITIVE', 'alcohol', 'FPRP', 'NEGATIVE'],
                    ['DT', 'NNP', 'IN', 'alcohol'],
                    ['FPRP', 'VBD', 'TO', 'alcohol', 'FPRP', 'TO', 'NEGATIVE'],
                    ['FPRP', 'VBD', 'alcohol'],
                    ['FPRP', 'alcohol', 'JJ', 'FPRP', 'MD', 'NEG', 'VB', 'RP'],
                    ['DT', 'NN', 'IN', 'DT', 'NEGATIVE', 'CC', 'alcohol'],
                    ['FPRP', 'VBP', 'INTSFR', 'VBN', 'RB', 'alcohol'],
                    ['FPRP', 'MD', 'VB', 'INTSFR', 'alcohol'],
                    ['SPRP', 'VBD', 'IN', 'alcohol'],
                    ['FPRP', 'alcohol', 'DT', 'NN'],
                ]



StrveSeqTags = [
                    ['FPRP','VBD','TO','starve'],
                    ['FPRP', 'VBD', 'TO', 'starve', 'FPRP', 'TO', 'NEGATIVE'],
                    ['starve', 'WDT', 'SWR', 'TO', 'NEGATIVE']
               ]

CMSeqTags = [
                ['FPRP','VBD','TO','carbon monoxide','FPRP','IN','NN','carbon monoxide'],
                ['FPRP', 'VBD', 'TO', 'carbon monoxide'],
                ['FPRP','CC','INTSFR','IN','IN','SPRP','IN','VBG','CC','carbon monoxide'],
                ['FPRP','IN','NN','carbon monoxide']
            ]

ElecSeqTags = [
                    ['FPRP','VBD','TO','electrocute'],
                    ['electrocute', 'IN', 'DT', 'NNS'],
                    ['FPRP', 'VBD', 'TO', 'electrocute', 'FPRP']
              ]

DisSeqTags =   [
                    ['FPRP','VBP','NEGATIVE',',','NEGATIVE',',','CC','INTSFR','NEGATIVE','JJ','disease'],
                    ['FPRP', 'VBD', 'VBN', 'IN', 'NNP', 'NNP', 'disease'],
                    ['FPRP', 'VBP', 'VBN', 'IN', 'disease'],
                    ['FPRP', 'POSITIVE', 'IN', 'disease'],
                    ['FPRP', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'disease'],
                    ['FPRP', 'VBP', 'VBN', 'IN', 'disease'],
                    ['FPRP', 'RB', 'NEGATIVE', 'IN', 'disease'],
                    ['DT', 'NEGATIVE', 'disease'],
                    ['INTSFR', 'NEGATIVE', 'JJ', 'disease'],
                    ['NEGATIVE', 'JJ', 'disease'],
                    ['FPRP', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'disease'],
                    ['DT', 'JJ', 'disease'],
                    ['FPRP', 'JJ', 'disease'],
                    ['DT', 'NN', 'NEGATIVE', 'NEGATIVE', 'disease']
                ]

DrwnSeqTags =   [
                    ['FPRP', 'VBD', 'NEG', 'drown'],
                    ['FPRP', 'VBP', 'VBN', 'drown'],
                    ['FPRP', 'VBD', 'RB', 'CD', 'NNS', 'CC', 'VBD', 'TO', 'drown'],
                    ['DT', 'NN', 'VBD', 'TO', 'drown', 'FPRP'],
                    ['TO', 'drown', 'FPRP'],
                    ['NNP', 'TO', 'drown', 'FPRP'],
                    ['FPRP', 'VBD', 'TO', 'drown', 'FPRP'],
                    ['VBD', 'TO', 'drown', 'FPRP'],
                    ['FPRP', 'MD', 'NEG', 'VB', 'FPRP', 'drown'],
                    ['JJ', 'drown', 'FPRP'],
                    ['FPRP', 'drown', 'NEGATIVE']
                ]


HangSeqTags =   [
                    ['RB', 'hang', 'FPRP'],
                    ['FPRP', 'hang', 'FPRP'],
                    ['FPRP', 'VBD', 'INTSFR', 'VBN', 'TO', 'NEGATIVE', 'FPRP'],
                    ['NEGATIVE', 'NN', 'hang', 'FPRP', 'VBD', 'VBG'],
                    ['DT', 'hang', 'NEGATIVE', 'CC', 'FPRP', 'VBD', 'RB', 'RB'],
                    ['DT', 'hang', 'NEGATIVE'],
                    ['DT', 'hang', 'NEGATIVE'],
                    ['DT', 'NEGATIVE', 'hang'],
                    ['FPRP', 'JJ', 'hang'],
                    ['FPRP', 'hang', 'DT', 'JJ', 'NN'],
                    ['FPRP', 'NEGATIVE', 'DT', 'hang'],
                    ['FPRP', 'NEGATIVE', 'DT', 'hang'],
                    ['DT', 'NN', 'hang'],
                    ['DT', 'hang', 'NEGATIVE'],
                    ['TO', 'hang', 'CC'],
                    ['FPRP', 'VBD', 'VBG', 'IN', 'DT', 'hang'],
                    ['FPRP', 'VBP', 'VBN', 'hang'],
                    ['FPRP','DT','VBP','RP','IN','DT','hang'],
                    ['FPRP', 'VBD', 'TO', 'hang'],
                    ['FPRP', 'VBD', 'DT', 'hang'],
                    ['FPRP', 'VBD', 'hang'],
                    ['FPRP', 'VBD', 'IN', 'IN', 'JJ', 'NNS'],
                    ['FPRP', 'VBD', 'VBN', 'TO', 'hang'],
                    ['FPRP', 'JJ', 'NN', 'VBD', 'IN', 'DT', 'hang'],
                    ['FPRP', 'VBD', 'VBN', 'hang'],
                    ['FPRP', 'VBD', 'DT', 'VBG', 'NN', 'hang'],
                    ['FPRP','NEGATIVE','TO','VB','JJ','NNP','NNP','VBD','WRB','DT','hang'],
                    ['FPRP', 'hang', 'IN', 'IN', 'JJ', 'NNS'],
                    
                ]

BldSeqTags =   [
                    ['FPRP', 'VBD', 'IN', 'IN', 'SPRP', 'CC', 'NEGATIVE', 'VBD', 'DT', 'bleed'],
                    ['FPRP', 'VBD', 'VBG', 'TO', 'bleed'],
                    ['FPRP', 'MD', 'RB', 'bleed'],
                    ['FPRP', 'VBD', 'NEGATIVE', 'CC', 'bleed'],
                    ['FPRP', 'VBP', 'TO', 'VB', 'RP', 'CC', 'NEGATIVE', 'DT', 'bleed'],
                    ['FPRP', 'VBD', 'VBG', 'TO', 'bleed', 'TO', 'NEGATIVE'],
                    ['VBD', 'NEG', 'VBG', 'TO', 'VB', 'FPRP', 'NEGATIVE', 'FPRP'],
                    ['FPRP', 'VBP', 'TO', 'VB', 'RP', 'CC', 'NEGATIVE', 'DT', 'bleed'],
                    ['DT', 'bleed', 'IN', 'FPRP', 'NN', 'MD', 'VB', 'VBN', 'FPRP'],
                    ['VBD', 'DT', 'bleed'],
                    [ 'IN', 'bleed', 'NEGATIVE'],
                    ['CC', 'bleed', 'NEGATIVE'],
                    ['RB', 'bleed', 'RP'],
                    ['DT', 'bleed', 'NN'],
                    ['DT', 'bleed', 'IN']
                ]

DrugSeqTags =   [
                    ['FPRP', 'VBD', 'RB', 'CD', 'CC', 'INTSFR', 'JJ', 'drug'],
                    ['FPRP', 'drug', 'IN', 'NN'],
                    ['FPRP', 'VBD', 'INTSFR', 'VBN', 'JJ', 'drug'],
                    ['FPRP', 'VBP', 'VBN', 'DT', 'NN', 'IN', 'drug'],
                    ['IN', 'JJ', 'drug', 'IN', 'JJ'],
                    ['FPRP', 'VBP', 'FPRP', 'DT', 'JJS', 'drug'],
                    ['VBD', 'RP', 'DT', 'drug'],
                    ['FPRP', 'VBD', 'DT', 'POSITIVE', 'NN', 'drug'],
                    ['FPRP', 'VBD', 'DT', 'POSITIVE', 'drug'],
                    ['DT', 'NN', 'IN', 'drug'],
                    ['TO', 'VB', 'DT', 'drug'],
                    ['CD', 'NN', 'IN', 'drug'],
                    ['IN', 'VBG', 'PDT', 'DT', 'drug'],
                    ['VBG', 'DT', 'drug'],
                    ['VBG', 'TO', 'drug'],
                    ['FPRP', 'VBD', 'DT', 'NN', 'IN', 'drug'],
                    ['FPRP', 'drug', 'IN', 'CD', 'drug'],
                    ['VBD', 'DT', 'drug'],
                    ['FPRP', 'VBD', 'DT', 'POSITIVE', 'NN', 'IN', 'drug'],
                    ['FPRP', 'VBD', 'RP', 'DT', 'NN', 'IN', 'NEGATIVE', 'drug'],
                    ['JJ', 'NEGATIVE', 'IN', 'drug'],
                    ['FPRP', 'VBD', 'RB', 'JJ', 'drug', 'IN', 'FPRP', 'MD'],
                    ['VBN', 'TO', 'drug'],
                    ['FPRP', 'VBD', 'TO', 'drug', 'IN', 'drug'],
                    ['FPRP', 'VBD', 'CD', 'drug'],
                    ['VBG', 'DT', 'drug'],
                    ['FPRP', 'VBD', 'VBG', 'POSITIVE', 'CD', 'drug'],
                    ['NNS', 'IN', 'FPRP', 'drug'],
                    ['FPRP', 'MD', 'drug', "''", 'NN'],
                    ['FPRP', 'VBD', 'VBG', 'drug'],
                    ['FPRP', 'drug', 'IN', 'NN', 'drug'],
                    ['FPRP', 'VBD', 'IN', 'drug'],
                    ['FPRP', 'VBD', 'VBN', 'TO', 'drug'],
                    ['FPRP', 'VBD', 'TO', 'drug'],
                    ['FPRP', 'VBD', 'drug'],
                    ['FPRP', 'VBP', 'RB', 'IN', 'drug'],
                    ['FPRP', 'VBD', 'DT', 'drug'],
                    ['FPRP', 'VBD', 'CD', 'drug'],
                    ['FPRP', 'VBP', 'DT', 'IN', 'IN', 'drug'],
                    ['FPRP', 'VBD', 'DT', 'RB', 'JJ', 'drug'],
                    ['FPRP', 'VBP', 'DT', 'NEGATIVE', 'TO', 'DT', 'drug'],
                    ['VBD', 'IN', 'DT', 'drug', 'FPRP', 'VBD']
                ]

MethodSeqTags = [['FPRP', 'VBD', 'TO', 'METHOD'],
                 ['FPRP', 'VBD', 'DT', 'METHOD'],
                 ['FPRP', 'VBD', 'METHOD'],
                 ['FPRP', 'JJ', 'METHOD'],
                 ['FPRP', 'VBD', 'VBG', 'TO', 'METHOD'],
                 ['FPRP', 'VBD', 'VBN', 'TO', 'METHOD'],
                 ['FPRP', 'VBP', 'DT', 'METHOD'],
                 ['FPRP', 'VBD', 'DT', 'JJ', 'METHOD'],
                 ['FPRP', 'TO', 'METHOD'],
                 ['FPRP', 'POSITIVE', 'METHOD'],
                 ['FPRP', 'MD', 'METHOD'],
                 ['FPRP', 'IN', 'METHOD'],
                 ['FPRP', 'VBP', 'VBN', 'METHOD'],
                 ['FPRP', 'VBD', 'DT', 'NEGATIVE', 'METHOD'],
                 ['FPRP', 'VBD', 'VBN', 'DT', 'METHOD'],
                 ['FPRP', 'VBP', 'METHOD'],
                 ['FPRP', 'VBP', 'RB', 'IN', 'METHOD'],
                 ['FPRP', 'VBD', 'RP', 'DT', 'METHOD'],
                 ['FPRP', 'MD', 'VB', 'TO', 'METHOD'],
                 ['FPRP', 'NN', 'METHOD'],
                 ['FPRP', 'RB', 'NEGATIVE', 'IN', 'METHOD'],
                 ['FPRP', 'NN', 'IN', 'METHOD'],
                 ['FPRP', 'VBD', 'RP', 'VBG', 'DT', 'METHOD'],
                 ['FPRP', 'VBD', 'JJ', 'TO', 'METHOD'],
                 ['FPRP', 'TO', 'VB', 'DT', 'METHOD'],
                 ['FPRP', 'NN', 'IN', 'DT', 'METHOD'],
                 ['FPRP', 'VBD', 'VBG', 'DT', 'JJ', 'METHOD'],
                 ['FPRP', 'NN', 'VBD', 'TO', 'METHOD'],
                 ['FPRP', 'NNS', 'IN', 'DT', 'METHOD'],
                 ['FPRP', 'VBD', 'VBN', 'DT', 'NN', 'IN', 'METHOD']
                ]



#### Function to get the sentence length as a feature

In [61]:
def lenthresholdSent(sent):

    if(len(sent)> 5):
        sentlength = '+5'
    else:
        sentlength = '-5'
    return sentlength

#### Function to count the number of times a sequence of tag occurs in a particular sentences

In [62]:
def get_count(sentence, seq_tags):
    #seq_tags = ['FPRP', 'VB', 'INTSFR','NEG']
    #sentence= [('I', 'FPRP'), ('felt', 'VBD'), ('embarrassed', 'VBN'), ('.', '.')]
    tag_order = []
    count = 0
    for i, element in enumerate(sentence):
        if len(tag_order) < len(seq_tags):
            tag_order = tag_order + [element[1]]
            if (len(tag_order) == len(seq_tags)) and (seq_tags == tag_order):
                count = 1
        else:
            if seq_tags == tag_order:
                count = 1
                j = i - (len(seq_tags) - 1) - 1
                """while j < i :
                    print(sentence[j][0], end=" ")
                
                    j += 1
                print()"""
            tag_order.pop(0)
            tag_order = tag_order + [element[1]]
    return count

#### Function to iterate over all the rules for a Method and query the get_count method to count the number of times it occurs in the sentence

In [63]:
def count_seq(Tags, i):
    Count = 0
    for indexsev, seqtg in enumerate(Tags):
        if(len(seqtg)==1):
            if len(sentences_ourtag[i])==1:
                Count += get_count(sentences_ourtag[i], seqtg)
        else:
            Count += get_count(sentences_ourtag[i], seqtg)
    return Count

#### Getting all the features required in CRF Format

In [64]:
# This is the format of the scored_sentences which is a set of features for sentences
"""
[posts[i], cfprp, csprp, cneg, cablt, cswr, cintfr, cnegative, cnegintfr, flag, cpositive, cwprp, senti_Vader[i], int(senti_Afinn[i]), cmethod]
"""
# An example of scored_sentences
"""
['Well I had attempted to hang myself when I was 15.', 3, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]"""

## features to use
freqfeatures = True # If we want to use the freqfeatures 
uselengthsentfeature = False # If we want to use the sentence length as a feature
#useSevforAllClasses = True # 
colon_features = True # If we want to use the colon as a feature

def noseqsent2features_tupleseq(seq, i):   

    sent = seq[0]
    sentlength = lenthresholdSent(nltk.word_tokenize(sent))
    
    features = []
    
    if(uselengthsentfeature): 
        features.append('sent.length=' + sentlength)
   
    # Test for the rules : 
    drugCount = count_seq(DrugSeqTags, i)
    hangCount = count_seq(HangSeqTags, i)
    gunCount = count_seq(GunSeqTags, i)
    cutCount = count_seq(CutSeqTags, i)
    alcoholCount = count_seq(AlcSeqTags, i)
    jmpCount = count_seq(JumpSeqTags, i)
    drwnCount = count_seq(DrwnSeqTags, i)
    bldCount = count_seq(BldSeqTags, i)
    vhlCount = count_seq(VehSeqTags, i)
    cmCount = count_seq(CMSeqTags, i)
    disCount = count_seq(DisSeqTags, i)
    strvCount = count_seq(StrveSeqTags, i)
    ectCount = count_seq(ElecSeqTags, i)
    
    # Checking if the count for each sequence of rules is more than one
    # Used for the colon features
    dg, hg, gn, ct, acl, jmp, drwn, bld, vhl, cm, dis, strv, ect = [1 if x >= 1 else 0 for x in (drugCount, hangCount, 
                                                                    gunCount, cutCount, alcoholCount, jmpCount, drwnCount, 
                                                                    bldCount, vhlCount, cmCount, disCount, strvCount, ectCount)]
    
    # If we are using frequencies of count of first personal pronoun, etc 
    f1, f2, f3, f4, f5, f6, f7, f8, f10, f11, f12, f13, f14 = [1 if x != 0 else 0 for x in (seq[1], seq[2], seq[3], seq[4], seq[5]
                                                                            , seq[6], seq[7], seq[8], seq[10], seq[11],
                                                                           seq[12], seq[13], seq[14])]
    
    
    # Checking if any of the sequences have occurred in the sentence. If these sequences haven't occurred we randomly search for 
    # words related to any of the 16 methods and we count the frequency of the words and give that frequency as a feature to 
    # CRF...
    
    if dg== hg== gn== ct== acl== jmp== drwn== bld== vhl== cm== dis== strv== ect== 0: # No sequence rules satisfied
        for word in sentences_ourtag[i]:
            temp_word = wordnet_lemmatizer.lemmatize(word[0].lower(), get_wordnet_pos(nltk.pos_tag([word[0].lower()])[0][1]))
            if temp_word.lower() in synonym_dict: # Checking if word is present in method dictionary
                if synonym_dict[temp_word.lower()] == 'hang':
                    hg += 1
                elif synonym_dict[temp_word.lower()] == 'gun':
                    gn += 1
                elif synonym_dict[temp_word.lower()] == 'drug':
                    dg += 1
                elif synonym_dict[temp_word.lower()] == 'cut':
                    ct += 1
                elif synonym_dict[temp_word.lower()] == 'alcohol':
                    acl += 1
                elif synonym_dict[temp_word.lower()] == 'jump':
                    jmp += 1
                elif synonym_dict[temp_word.lower()] == 'drown':
                    drwn += 1
                elif synonym_dict[temp_word.lower()] == 'bleed':
                    bld += 1
                elif synonym_dict[temp_word.lower()] == 'vehicle':
                    vhl += 1
                elif synonym_dict[temp_word.lower()] == 'carbon monoxide':
                    cm += 1
                elif synonym_dict[temp_word.lower()] == 'disease':
                    dis += 1
                elif synonym_dict[temp_word.lower()] == 'starve':
                    strv += 1
                elif synonym_dict[temp_word.lower()] == 'electrocute':
                    ect += 1
        # if No sequence rules satisfied and no words are seen related to methods
        if dg== hg== gn== ct== acl== jmp== drwn== bld== vhl== cm== dis== strv== ect== 0: 
                # Consider adding the features below if none of the features above have satisfied
                # Add these features if we require to add features of Count of first personal pronoun, intensifiers, negative words, etc
                if colon_features: # If we want to use Colon Features
                    features.append('freq.cfprp=' + str(f1)+':'+ str(seq[1])) # Count of First Person Pronouns
                    features.append('freq.csprp=' + str(f2)+':'+ str(seq[2])) # Count of Second Person Pronouns
                    features.append('freq.cneg=' + str(f3)+':'+ str(seq[3])) # Count of Negation words
                    features.append('freq.cablt=' + str(f4)+':'+ str(seq[4])) # Count of Ability Words like can, handle, support
                    features.append('freq.cswr=' + str(f5)+':'+ str(seq[5])) # Count of Swear words
                    features.append('freq.cintfr=' + str(f6)+':'+ str(seq[6])) # Count of intensifier Words
                    features.append('freq.cnegative=' + str(f7)+':'+ str(seq[7])) # Count of Negative Words
                    features.append('freq.cnegintfr=' + str(f8)+':'+ str(seq[8])) # Count of Negative Intensifier Words
                    features.append('freq.posemo=' + str(seq[9])) # Count of Positive Emotion Words like LOL, LMAO, etc. uses regular Expression
                    features.append('freq.cpositive=' + str(f10)+':'+ str(seq[10])) # Count of Positive Words
                    features.append('freq.cwprp=' + str(f11)+':'+ str(seq[11])) # Count of First Personal Pronouns like we, our, etc
                    features.append('freq.sentiVader=' + str(f12)+':'+ str(seq[12])) # Sentiment Score from the Vader Lexicon
                    features.append('freq.sentiAfinn=' + str(f13)+':'+ str(seq[13])) # Setiment Score of the Afinn Lexicon
                    features.append('freq.method=' + str(f14)+':'+ str(seq[14])) # Count of Number of Method Related Words  
                else:
                    features.append('freq.cfprp='+ str(seq[1])) # Count of First Person Pronouns
                    features.append('freq.csprp='+ str(seq[2])) # Count of Second Person Pronouns
                    features.append('freq.cneg=' + str(seq[3])) # Count of Negation words
                    features.append('freq.cablt='+ str(seq[4])) # Count of Ability Words like can, handle, support
                    features.append('freq.cswr=' + str(seq[5])) # Count of Swear words
                    features.append('freq.cintfr='+ str(seq[6])) # Count of intensifier Words
                    features.append('freq.cnegative='+ str(seq[7])) # Count of Negative Words
                    features.append('freq.cnegintfr='+ str(seq[8])) # Count of Negative Intensifier Words
                    features.append('freq.posemo=' + str(seq[9])) # Count of Positive Emotion Words like LOL, LMAO, etc. uses regular Expression
                    features.append('freq.cpositive='+ str(seq[10])) # Count of Positive Words
                    features.append('freq.cwprp=' + str(seq[11])) # Count of First Personal Pronouns like we, our, etc
                    features.append('freq.sentiVader='+ str(seq[12])) # Sentiment Score from the Vader Lexicon
                    features.append('freq.sentiAfinn='+ str(seq[13])) # Setiment Score of the Afinn Lexicon
                    features.append('freq.method=' + str(seq[14])) # Count of Number of Method Related Words  
        else:
                # Adding all the frequencies of words related to methods to the CRF Feature Set
                features.append('DrugFreq=' +str(dg))
                features.append('HangFreq=' +str(hg))
                features.append('GunFreq=' +str(gn))
                features.append('CutFreq=' +str(ct))
                features.append('AlcoholFreq=' +str(acl))
                features.append('JumpFreq=' +str(jmp))
                features.append('DrownFreq=' +str(drwn))
                features.append('BleedFreq=' +str(bld))
                features.append('VehicleFreq=' +str(vhl))
                features.append('MonoxideFreq=' +str(cm))
                features.append('DiseaseFreq=' +str(dis))
                features.append('StarveFreq=' +str(strv))
                features.append('ElectrocuteFreq=' +str(ect))
    else:
        # Adding the sequence frequency occurrence to the CRF Feature Set
        if colon_features: # If we want to use Colon Features
            features.append('Drug=' +str(dg)+':'+ str(drugCount))
            features.append('Hang=' +str(hg)+':'+ str(hangCount))
            features.append('Gun=' +str(gn)+':'+ str(gunCount))
            features.append('Cut=' +str(ct)+':'+ str(cutCount))
            features.append('Alcohol=' +str(acl)+':'+ str(alcoholCount))
            features.append('Jump=' +str(jmp)+':'+ str(jmpCount))
            features.append('Drown=' +str(drwn)+':'+ str(drwnCount))
            features.append('Bleed=' +str(bld)+':'+ str(bldCount))
            features.append('Vehicle=' +str(vhl)+':'+ str(vhlCount))
            features.append('Monoxide=' +str(cm)+':'+ str(cmCount))
            features.append('Disease=' +str(dis)+':'+ str(disCount))
            features.append('Starve=' +str(strv)+':'+ str(strvCount))
            features.append('Electrocute=' +str(ect)+':'+ str(ectCount))
        else:   # If we don't want to use Colon Features
            features.append('Drug=' + str(drugCount))
            features.append('Hang=' + str(hangCount))
            features.append('Gun=' + str(gunCount))
            features.append('Cut=' + str(cutCount))
            features.append('Alcohol=' + str(alcoholCount))
            features.append('Jump=' + str(jmpCount))
            features.append('Drown=' + str(drwnCount))
            features.append('Bleed=' + str(bldCount))
            features.append('Vehicle=' + str(vhlCount))
            features.append('Monoxide=' + str(cmCount))
            features.append('Disease=' + str(disCount))
            features.append('Starve=' + str(strvCount))
            features.append('Electrocute=' + str(ectCount))
    
    #features.append('EOS')
    return features


#give me a sentences with the frequencies as input : seq = sent, 2, 1 , ...
def noseq2features_tupleseq(seq,index):
    return [noseqsent2features_tupleseq(seq, index)]

def noseq2labels(seq):
    return [label for label in seq]

#### Calling the function to extract the features from the sentences

In [66]:
train_data = [noseq2features_tupleseq(s,index) for index,s in enumerate(scored_sentences)]
train_labels = [[label]  for label in allannotations.annotation] 

In [67]:
train_data

[[['Drug=0:0',
   'Hang=1:1',
   'Gun=0:0',
   'Cut=0:0',
   'Alcohol=0:0',
   'Jump=0:0',
   'Drown=0:0',
   'Bleed=0:0',
   'Vehicle=0:0',
   'Monoxide=0:0',
   'Disease=0:0',
   'Starve=0:0',
   'Electrocute=0:0']],
 [['Drug=0:0',
   'Hang=1:4',
   'Gun=0:0',
   'Cut=0:0',
   'Alcohol=0:0',
   'Jump=0:0',
   'Drown=0:0',
   'Bleed=0:0',
   'Vehicle=0:0',
   'Monoxide=0:0',
   'Disease=0:0',
   'Starve=0:0',
   'Electrocute=0:0']],
 [['Drug=0:0',
   'Hang=1:1',
   'Gun=0:0',
   'Cut=0:0',
   'Alcohol=0:0',
   'Jump=0:0',
   'Drown=0:0',
   'Bleed=0:0',
   'Vehicle=0:0',
   'Monoxide=0:0',
   'Disease=0:0',
   'Starve=0:0',
   'Electrocute=0:0']],
 [['Drug=0:0',
   'Hang=1:1',
   'Gun=0:0',
   'Cut=0:0',
   'Alcohol=0:0',
   'Jump=0:0',
   'Drown=0:0',
   'Bleed=0:0',
   'Vehicle=0:0',
   'Monoxide=0:0',
   'Disease=0:0',
   'Starve=0:0',
   'Electrocute=0:0']],
 [['Drug=0:0',
   'Hang=1:4',
   'Gun=0:0',
   'Cut=0:0',
   'Alcohol=0:0',
   'Jump=0:0',
   'Drown=0:0',
   'Bleed=0:0',
 

In [68]:
train_labels

[['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['drug'],
 ['alcohol'],
 ['alcohol'],
 ['alcohol'],
 ['alcohol'],
 ['drug'],
 ['gun'],
 ['disease'],
 ['gun'],
 ['gun'],
 ['drug'],
 ['jump'],
 ['vehicle'],
 ['gun'],
 ['gun'],
 ['drug'],
 ['drug'],
 ['drug'],
 ['drug'],
 ['alcohol'],
 ['drug'],
 ['alcohol'],
 ['drug'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['cut'],
 ['bleed'],
 ['hang'],
 ['cut'],
 ['hang'],
 ['drug'],
 ['hang'],
 ['cut'],
 ['bleed'],
 ['cut'],
 ['bleed'],
 ['drug'],
 ['drug'],
 ['drug'],
 ['cut'],
 ['drug'],
 ['drug'],
 ['cut'],
 ['hang'],
 ['hang'],
 ['gun'],
 ['gun'],
 ['hang'],
 ['hang'],
 ['jump'],
 ['jump'],
 ['jump'],
 ['gun'],
 ['hang'],
 ['cut'],
 ['cut'],
 ['cut'],
 ['drug'],
 ['drug'],
 ['cut'],
 ['alcohol'],
 ['drug'],
 ['gun'],
 ['gun'],
 ['gun'],
 ['gun'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['hang'],
 ['drug'],
 ['gun'],
 ['gun'],
 ['hang'],
 ['drug'],
 ['gun'],
 ['gun'],
 ['sta

#### Counting number of Labels of each type in the training set

In [70]:
nested_lst_of_tuples = [tuple(l) for l in train_labels]
count = Counter(elem for elem in nested_lst_of_tuples)
sorted(count.items(), key=lambda x: x[1], reverse = True)

[(('drug',), 210),
 (('hang',), 90),
 (('gun',), 49),
 (('alcohol',), 47),
 (('cut',), 39),
 (('jump',), 23),
 (('disease',), 13),
 (('bleed',), 9),
 (('drown',), 8),
 (('vehicle',), 8),
 (('starve',), 2),
 (('carbon monoxide',), 2),
 (('electrocute',), 2)]

In [71]:
validation_size = 0.3
seed = 7
#lb=preprocessing.MultiLabelBinarizer()
#y = lb.fit_transform(train_labels)

X_train, X_validation, Y_train, Y_validation = train_test_split(train_data, train_labels, 
                                                                                 test_size=validation_size, 
                                                                                 random_state=seed, stratify = train_labels
                                                                                )

#### Fitting the CRF Model

In [72]:
%%time
#
len(X_train)
#
len(Y_train)
#############################################
# CRF params 
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.6, 
    c2=0.7, 
    max_iterations=2000, 
    all_possible_transitions=True,
    all_possible_states=True
)
crf.fit(X_train, Y_train)


labels = list(crf.classes_)
#labels.remove('111')
labels

Wall time: 562 ms


#### Performing Cross Validation

In [73]:
cv = 5
#cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
scores = cross_val_score(crf, X_train, Y_train, cv=cv, scoring='f1_macro')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average

Accuracy: 0.60 (+/- 0.26)


#### Calculating F1 Score and Accuracy

In [74]:
%%time

Y_pred = crf.predict(X_validation)
Y_pred


print('F1 score : ', metrics.flat_f1_score(Y_validation, Y_pred, 
                      average='weighted', labels=labels))

print('Accuracy score : ', metrics.flat_accuracy_score(Y_validation, Y_pred))

C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


F1 score :  0.808500208821
Accuracy score :  0.82119205298
Wall time: 101 ms


#### Getting the Classification Report

In [75]:
# group 1 and 2 results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)

sorted_labels
#
print(metrics.flat_classification_report(
    Y_validation, Y_pred, labels=sorted_labels, digits=3
))

C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                 precision    recall  f1-score   support

           hang      0.897     0.963     0.929        27
carbon monoxide      0.000     0.000     0.000         1
        vehicle      0.000     0.000     0.000         2
        disease      1.000     0.750     0.857         4
        alcohol      0.667     0.571     0.615        14
    electrocute      0.000     0.000     0.000         1
          bleed      0.000     0.000     0.000         3
          drown      0.500     0.500     0.500         2
           drug      0.851     0.905     0.877        63
         starve      0.000     0.000     0.000         0
           jump      0.500     0.857     0.632         7
            gun      1.000     0.933     0.966        15
            cut      0.900     0.750     0.818        12

    avg / total      0.804     0.821     0.809       151



#### Calculating micro F1 score and Accuracy

In [76]:
#print accuracy_score
print ("accuracy : "+str(accuracy_score(Y_validation, Y_pred)))
print ("micro f-measure "+str(f1_score(Y_validation, Y_pred, average='micro')))
print ("macro f-measure "+str(f1_score(Y_validation, Y_pred, average='macro')))

accuracy : 0.82119205298
micro f-measure 0.82119205298


C:\Users\AshwinAmbal\Anaconda3\envs\intern\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


macro f-measure 0.516108332079


#### Code used to find which labels were wrongly Labeled

In [77]:
i= 0
for s, x, y in zip(scored_sentences, train_data, train_labels) : 
    sentence = [l[1] for l in sentences_ourtag[i]]
    prediction = crf.predict([x])
    if  y[0] == 'bleed' and prediction[0][0] != 'bleed':
        print(i,s[0],'\n', sentence,'\n', x, y, prediction)
        print("<==============END===============>")
    i+=1

44 The first time I was rather young, so when I cut myself and did not bleed, I freaked out and decided ''the universe'' was not going to let me kill myself. 
 ['DT', 'JJ', 'NN', 'FPRP', 'VBD', 'INTSFR', 'POSITIVE', ',', 'INTSFR', 'WRB', 'FPRP', 'cut', 'FPRP', 'CC', 'VBD', 'NEG', 'bleed', ',', 'FPRP', 'VBD', 'RP', 'CC', 'VBD', 'PRP', 'NN', "''", 'VBD', 'NEG', 'VBG', 'TO', 'VB', 'FPRP', 'NEGATIVE', 'FPRP', '.'] 
 [['Drug=0:0', 'Hang=0:0', 'Gun=0:0', 'Cut=1:2', 'Alcohol=0:0', 'Jump=0:0', 'Drown=0:0', 'Bleed=1:1', 'Vehicle=0:0', 'Monoxide=0:0', 'Disease=0:0', 'Starve=0:0', 'Electrocute=0:0']] ['bleed'] [['cut']]
<==============END===============>
46 Turns out, you can totally cut yourself and not bleed for a variety of reasons; but I thought I had tried to cheat some divine plan and not bleeding out was the cosmos saying 'nice try, but not today'". 
 ['VBZ', 'RP', ',', 'SPRP', 'ABLT', 'INTSFR', 'cut', 'SPRP', 'CC', 'NEG', 'bleed', 'IN', 'DT', 'POSITIVE', 'IN', 'NNS', ':', 'CC', 'FPRP', 'V

#### Code used to find which features contribute most to predicting a particular label

In [78]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    
#
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))
#
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])


Top positive:
1.710679 alcohol  AlcoholFreq=1
1.524058 alcohol  Alcohol=1:1
1.488894 gun      GunFreq=1
1.304814 gun      Gun=1:1
1.253568 jump     JumpFreq=1
1.172219 hang     Hang=1:1
1.066237 disease  Disease=1:1
1.037130 drug     Drug=1:2
0.931854 drug     DrugFreq=1
0.894736 drug     HangFreq=0
0.881713 hang     DrugFreq=0
0.877355 cut      Cut=1:1
0.824928 hang     HangFreq=1
0.823049 drown    Drown=1:3
0.772043 cut      CutFreq=1
0.770307 jump     Jump=1:2
0.767336 drug     JumpFreq=0
0.743146 jump     DrugFreq=0
0.716203 jump     Jump=1:1
0.712420 vehicle  Jump=1:1
0.711426 drug     Drug=1:1
0.620489 vehicle  Vehicle=1:1
0.615249 drug     GunFreq=0
0.592843 hang     HangFreq=2
0.589285 drug     Gun=0:0
0.572113 drug     VehicleFreq=0
0.560633 hang     JumpFreq=0
0.555061 drug     Jump=0:0
0.541776 hang     AlcoholFreq=0
0.512313 drug     DrugFreq=2

Top negative:
-0.165112 electrocute Monoxide=0:0
-0.171711 jump     DrugFreq=1
-0.173166 drug     GunFreq=1
-0.183792 alcohol  Dru

#### Code to tune the Classifier to get the best parameters to improve accuracy and f1 score

In [33]:
"""params_space = {
    'c1': scipy.stats.expon(scale=0.05),
    'c2': scipy.stats.expon(scale=0.05)
    
    #'algorithm' : ('lbfgs', 'ap','pa',),
}

#
# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

#
# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=5, 
                        verbose=1, 
                        n_jobs=5, 
                        n_iter=500, 
                        scoring=f1_scorer)

# 
rs.fit(X_train, Y_train)"""

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   31.4s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  1.3min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:  2.7min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed:  4.8min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed:  7.2min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed: 10.4min
[Parallel(n_jobs=5)]: Done 2440 tasks      | elapsed: 13.4min
[Parallel(n_jobs=5)]: Done 2500 out of 2500 | elapsed: 13.7min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=CRF(algorithm='lbfgs', all_possible_states=True,
  all_possible_transitions=True, averaging=None, c=None, c1=0.6, c2=0.7,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_s...e,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False),
          fit_params={}, iid=True, n_iter=500, n_jobs=5,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000B0F5A2D940>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000B0F5A2DC50>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True,
          scoring=make_scorer(flat_f1_score, average=weighted, labels=['hang', 'drug', 'alcohol', 'vehicle', 'gun', 'cut', 'jump', 'drown', 'starve', 'bleed', 'carbon monoxide', 'diseas